In [1]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
from sklearn.utils import shuffle

Using TensorFlow backend.
/home/luisselvera/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/luisselvera/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/luisselvera/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/luisselvera/anaconda3/lib/python3.7

In [25]:
import tensorflow as tf

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(1)

Found GPU at: /device:GPU:0


'GeForce GTX 1080 Ti'

In [3]:
#Reading in Non_Russians data set
Non_Russian=pd.read_csv("data/csv-zusammenfuehren.de_3mjg6fs7.csv", error_bad_lines=False)
#Reading in comments by Russian Trolls
Russian = pd.read_csv("data/comments.csv")

#Get only the comments
Non_Russian = Non_Russian[["body"]]
Russian = Russian[["body"]]

#Dropping rows with N/A
Non_Russian.dropna(inplace=True)
Russian.dropna(inplace=True)

num_samples = Russian.shape[0]
Non_Russian = Non_Russian.sample(num_samples)
# Non_Russian.sample(10)

b'Skipping line 4581: expected 20 fields, saw 22\nSkipping line 9684: expected 20 fields, saw 21\nSkipping line 11215: expected 20 fields, saw 27\n'
b'Skipping line 33342: expected 20 fields, saw 21\nSkipping line 38620: expected 20 fields, saw 22\nSkipping line 40411: expected 20 fields, saw 21\n'
/home/luisselvera/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (1,2,3,6) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
Non_Russian.head()

,body
23620,The Bible is not true. The Bible is not true....
28873,**PLEASE READ THIS MESSAGE IN ITS ENTIRETY BEF...
35573,Thanks for the review. :)
40347,hyperloop cant work though. one compromise in ...
19781,Out of interest... What would you need WINE fo...


In [5]:
# Create sentence and label lists
Russian_sentences = Russian.body.values
Non_Russian_sentences = Non_Russian.body.values

# We need to add special tokens at the beginning and end of each sentence for BERT to work properly
Russian_sentences = ["[CLS] " + sentence + " [SEP]" for sentence in Russian_sentences]
Non_Russian_sentences = ["[CLS] " + sentence + " [SEP]" for sentence in Non_Russian_sentences]

In [6]:
Non_Russian_sentences[0]

'[CLS] The Bible is not true.  The Bible is not true.  The Bible is not true.  The Bible is not true.  The Bible is not true.  The Bible is not true.  The Bible is not true.  The Bible is not true.  The Bible is not true.  The Bible is not true.  The Bible is not true.  The Bible is not true.  The Bible is not true.  The Bible is not true.  The Bible is not true.  The Bible is not true.  The Bible is not true.  The Bible is not true.  The Bible is not true.  The Bible is not true.  The Bible is not true.  The Bible is not true.  The Bible is not true.  The Bible is not true.  The Bible is not true.  The Bible is not true.  The Bible is not true.  The Bible is not true.  The Bible is not true.  The Bible is not true.  The Bible is not true.  The Bible is not true.  The Bible is not true.  The Bible is not true.  The Bible is not true.  The Bible is not true.  The Bible is not true.  The Bible is not true.  The Bible is not true.  The Bible is not true.  The Bible is not true.  The Bible

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Russian_tokenized_texts = [tokenizer.tokenize(sent) for sent in Russian_sentences]
Non_Russian_tokenized_texts = [tokenizer.tokenize(sent) for sent in Non_Russian_sentences]

print ("Tokenize the first sentence:")
print (Russian_tokenized_texts[0])

Tokenize the first sentence:
['[CLS]', 'a', 'hard', 'look', 'at', 'training', 'and', 'tactics', '"', '=', 'they', 'will', 'be', 'sent', 'more', '$', '$', '$', 'for', '"', 'training', '"', '[SEP]']


In [8]:
Russian_tokenized_texts = [x for x in Russian_tokenized_texts if len(x) < 512]
Non_Russian_tokenized_texts = [x for x in Non_Russian_tokenized_texts if len(x) < 512]

tokenized_texts = Non_Russian_tokenized_texts + Russian_tokenized_texts

In [9]:
len(tokenized_texts)

13374

In [10]:
#Adding Labels
Non_Russian_labels = [0 for x in range(len(Non_Russian_tokenized_texts))]
Russian_labels = [1 for x in range(len(Russian_tokenized_texts))]

labels = Non_Russian_labels + Russian_labels

In [11]:
len(labels)

13374

In [12]:
# Set the maximum sequence length. The longest sequence in our training set is 47, but we'll leave room on the end anyway. 
# In the original paper, the authors used a length of 512.
MAX_LEN = 512

In [13]:
# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

In [14]:
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [15]:
len(input_ids)

13374

In [16]:
# Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

In [17]:
# Use train_test_split to split our data into train and validation sets for training

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids, labels, random_state=2018, test_size=0.1)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids, random_state=2018, test_size=0.1)

In [18]:
# Convert all of our data into torch tensors, the required datatype for our model

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [19]:
# Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32
batch_size = 32

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [26]:
# Load BertForSequenceClassification, the pretrained BERT model with a single linear classification layer on top. 

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model.cuda()

RuntimeError: CUDA out of memory. Tried to allocate 90.00 MiB (GPU 1; 10.92 GiB total capacity; 10.07 GiB already allocated; 56.00 MiB free; 100.39 MiB cached)

In [21]:
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'gamma', 'beta']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]


In [22]:
# This variable contains all of the hyperparemeter information our training loop needs
optimizer = BertAdam(optimizer_grouped_parameters,
                     lr=2e-5,
                     warmup=.1)

t_total value of -1 results in schedule not being applied


In [23]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [24]:
t = [] 

# Store our loss and accuracy for plotting
train_loss_set = []

# Number of training epochs (authors recommend between 2 and 4)
epochs = 4
 
# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):


    # Training

    # Set our model to training mode (as opposed to evaluation mode)
    model.train()

    # Tracking variables
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0

    # Train the data for one epoch
    for step, batch in enumerate(train_dataloader):
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Clear out the gradients (by default they accumulate)
        optimizer.zero_grad()
        # Forward pass
        loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
        train_loss_set.append(loss.item())    
        # Backward pass
        loss.backward()
        # Update parameters and take a step using the computed gradient
        optimizer.step()


        # Update tracking variables
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1

    print("Train loss: {}".format(tr_loss/nb_tr_steps))


    # Validation
    model.eval()

    # Tracking variables 
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids, b_input_mask, b_labels = batch
        # Telling the model not to compute or store gradients, saving memory and speeding up validation
        with torch.no_grad():
          # Forward pass, calculate logit predictions
          logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        tmp_eval_accuracy = flat_accuracy(logits, label_ids)

        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 192.00 MiB (GPU 1; 10.92 GiB total capacity; 10.07 GiB already allocated; 56.00 MiB free; 100.39 MiB cached)